In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, normalize

In [29]:
chem = pd.read_csv(r"C:\Users\Administrator.DAI-PC2\Downloads\Shubham\Practical Machine Learning\ClassWork\Cases\Chemical Process Data\ChemicalProcess.csv")
y = chem["Yield"]
X = chem.drop("Yield",axis=1)

In [30]:
imp = SimpleImputer(strategy="mean").set_output(transform="pandas")
X_imp = imp.fit_transform(X)

In [31]:
X_imp.isnull().sum().sum()

0

In [32]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=24)

In [ ]:
# Linear regression

In [33]:
imp = SimpleImputer(strategy="mean").set_output(transform="pandas")
X_imp_trn = imp.fit_transform(X_train)
X_imp_tst = imp.transform(X_test)

In [35]:
lr = LinearRegression()
lr.fit(X_imp_trn, y_train)
y_pred = lr.predict(X_imp_tst)
print(r2_score(y_test, y_pred))

0.2393018585944332


In [62]:
## pipeline and linear regression

In [42]:
lr = LinearRegression()
imp = SimpleImputer(strategy="mean").set_output(transform="pandas")
pipe = Pipeline([("SimpleImp",imp),("Lr",lr)])

pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test, y_pred))

0.2393018585944332


In [43]:
## Gridsearch pipeline and linear regression

In [45]:
lr = LinearRegression()
imp = SimpleImputer(strategy="mean").set_output(transform="pandas")
pipe = Pipeline([("SimpleImp",imp),("Lr",lr)])

print(pipe.get_params())

{'memory': None, 'steps': [('SimpleImp', SimpleImputer()), ('Lr', LinearRegression())], 'verbose': False, 'SimpleImp': SimpleImputer(), 'Lr': LinearRegression(), 'SimpleImp__add_indicator': False, 'SimpleImp__copy': True, 'SimpleImp__fill_value': None, 'SimpleImp__keep_empty_features': False, 'SimpleImp__missing_values': nan, 'SimpleImp__strategy': 'mean', 'Lr__copy_X': True, 'Lr__fit_intercept': True, 'Lr__n_jobs': None, 'Lr__positive': False}


In [54]:
params = {"SimpleImp__strategy":["mean","median"]}
kfold = KFold(n_splits =5,shuffle = True, random_state = 24)
gcv = GridSearchCV(pipe,param_grid = params,cv=kfold, scoring="r2")
gcv.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('SimpleImp', SimpleImputer()),
                                       ('Lr', LinearRegression())]),
             param_grid={'SimpleImp__strategy': ['mean', 'median']},
             scoring='r2')

In [55]:
print(gcv.best_params_)
print(gcv.best_score_)

{'SimpleImp__strategy': 'mean'}
-8.218967417884183


In [56]:
pd.DataFrame(gcv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_SimpleImp__strategy,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011655,0.003771,0.004779,0.000399,mean,{'SimpleImp__strategy': 'mean'},0.452546,-6.223876,-0.099737,-35.477359,0.253589,-8.218967,13.855696,1
1,0.008764,0.001161,0.003783,0.000399,median,{'SimpleImp__strategy': 'median'},0.452680,-4.701617,0.353541,-38.290121,0.269335,-8.383236,15.081431,2


In [1]:
### Knn with imputer and pipeline and gridsearch

In [59]:
knn = KNeighborsRegressor()
std_scl = StandardScaler()
mm_scl = StandardScaler()
imp = SimpleImputer(strategy="mean").set_output(transform="pandas")
pipe = Pipeline([("SimpleImp",imp),("Scl",None),("Knn",knn)])

print(pipe.get_params())

{'memory': None, 'steps': [('SimpleImp', SimpleImputer()), ('Scl', None), ('Knn', KNeighborsRegressor())], 'verbose': False, 'SimpleImp': SimpleImputer(), 'Scl': None, 'Knn': KNeighborsRegressor(), 'SimpleImp__add_indicator': False, 'SimpleImp__copy': True, 'SimpleImp__fill_value': None, 'SimpleImp__keep_empty_features': False, 'SimpleImp__missing_values': nan, 'SimpleImp__strategy': 'mean', 'Knn__algorithm': 'auto', 'Knn__leaf_size': 30, 'Knn__metric': 'minkowski', 'Knn__metric_params': None, 'Knn__n_jobs': None, 'Knn__n_neighbors': 5, 'Knn__p': 2, 'Knn__weights': 'uniform'}


In [60]:
params = {"SimpleImp__strategy":["mean","median"],"Scl":[std_scl,mm_scl,None],"Knn__n_neighbors":np.arange(1,11)}
kfold = KFold(n_splits =5,shuffle = True, random_state = 24)
gcv = GridSearchCV(pipe,param_grid = params,cv=kfold, scoring="r2")
gcv.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('SimpleImp', SimpleImputer()),
                                       ('Scl', None),
                                       ('Knn', KNeighborsRegressor())]),
             param_grid={'Knn__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'Scl': [StandardScaler(), StandardScaler(), None],
                         'SimpleImp__strategy': ['mean', 'median']},
             scoring='r2')

In [61]:
print(gcv.best_params_)
print(gcv.best_score_)

{'Knn__n_neighbors': 3, 'Scl': StandardScaler(), 'SimpleImp__strategy': 'mean'}
0.4835363219733173
